In [1]:
import torch
# import numpy as np

In [2]:
torch.__version__

'2.1.2'

# `view()`, `reshape()`, `transpose()`, `permute()`


[Discussion of `contiguous`](https://stackoverflow.com/questions/26998223/what-is-the-difference-between-contiguous-and-non-contiguous-arrays/26999092#26999092)

In [3]:
sample_tensor = torch.rand(12)

In [4]:
sample_tensor

tensor([0.7871, 0.4773, 0.6763, 0.4647, 0.8857, 0.6221, 0.6647, 0.2766, 0.5620,
        0.4554, 0.6573, 0.7052])

In [5]:
sample_tensor.view(3, 4)

tensor([[0.7871, 0.4773, 0.6763, 0.4647],
        [0.8857, 0.6221, 0.6647, 0.2766],
        [0.5620, 0.4554, 0.6573, 0.7052]])

In [6]:
sample_tensor.reshape(3, 4)

tensor([[0.7871, 0.4773, 0.6763, 0.4647],
        [0.8857, 0.6221, 0.6647, 0.2766],
        [0.5620, 0.4554, 0.6573, 0.7052]])

## `reshape()` vs `view()`

`reshape` tries to return a `view` if possible, otherwise copies to data to a contiguous tensor and returns the `view` on it. ([Source](https://discuss.pytorch.org/t/difference-between-view-reshape-and-permute/54157/2))

`view()` works on contiguous tensors and `reshape()` works on non-contugous tensors (`contiguous()` + `view()`)

In [7]:
x = torch.arange(12).view(4, 3)

# View works on contiguous tensors
print(x.is_contiguous())
print(x)
print(x.view(-1))

True
tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])


In [8]:
y = x.permute(1, 0)

# Reshape works on non-contugous tensors (contiguous() + view)
print(y)
print(y.is_contiguous())

tensor([[ 0,  3,  6,  9],
        [ 1,  4,  7, 10],
        [ 2,  5,  8, 11]])
False


In [9]:
try:
    print(y.view(-1))
except RuntimeError as e:
    print(e)

view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.


In [10]:
print(y.reshape(-1))
print(y.contiguous().view(-1))

tensor([ 0,  3,  6,  9,  1,  4,  7, 10,  2,  5,  8, 11])
tensor([ 0,  3,  6,  9,  1,  4,  7, 10,  2,  5,  8, 11])


The usage of `view` and `reshape` does not depend on training / not-training.
I personally use `view` whenever possible and add a `contiguous` call to it, if necessary. This will make sure I see, where a copy is done in my code. `reshape` on the other hand does this automatically, so your code might look cleaner. [Source](https://discuss.pytorch.org/t/difference-between-view-reshape-and-permute/54157/4)

## `transpose()` vs `permute()` vs `swapaxes()`

`sawpaxes()` is an alias for `transpose()`. This function is equivalent to NumPy’s swapaxes function. [Source](https://pytorch.org/docs/stable/generated/torch.swapaxes.html#torch.swapaxes)

`permute()` and `tranpose()` are similar. `transpose()` can only swap two dimension. But `permute()` can swap all the dimensions.

In [11]:
x = torch.rand(2, 3, 4)

In [12]:
x, x.shape

(tensor([[[0.4428, 0.1552, 0.6599, 0.4228],
          [0.1577, 0.7957, 0.0523, 0.4348],
          [0.8402, 0.3298, 0.9236, 0.4066]],
 
         [[0.5747, 0.8458, 0.0487, 0.9425],
          [0.5154, 0.4294, 0.5961, 0.0803],
          [0.7357, 0.1801, 0.9983, 0.0248]]]),
 torch.Size([2, 3, 4]))

In [13]:
x.transpose(0, 2), x.transpose(0, 2).shape

(tensor([[[0.4428, 0.5747],
          [0.1577, 0.5154],
          [0.8402, 0.7357]],
 
         [[0.1552, 0.8458],
          [0.7957, 0.4294],
          [0.3298, 0.1801]],
 
         [[0.6599, 0.0487],
          [0.0523, 0.5961],
          [0.9236, 0.9983]],
 
         [[0.4228, 0.9425],
          [0.4348, 0.0803],
          [0.4066, 0.0248]]]),
 torch.Size([4, 3, 2]))

In [14]:
x.swapaxes(0, 2), x.swapaxes(0, 2).shape

(tensor([[[0.4428, 0.5747],
          [0.1577, 0.5154],
          [0.8402, 0.7357]],
 
         [[0.1552, 0.8458],
          [0.7957, 0.4294],
          [0.3298, 0.1801]],
 
         [[0.6599, 0.0487],
          [0.0523, 0.5961],
          [0.9236, 0.9983]],
 
         [[0.4228, 0.9425],
          [0.4348, 0.0803],
          [0.4066, 0.0248]]]),
 torch.Size([4, 3, 2]))

In [15]:
x.permute(2, 1, 0).shape

torch.Size([4, 3, 2])

In [16]:
(x.transpose(0, 2) == x.permute(2, 1, 0)).all()

tensor(True)

In [17]:
(x.transpose(0, 2) == x.swapaxes(0, 2)).all()

tensor(True)

In [18]:
(x.transpose(0, 2) == x.transpose(2, 0)).all()

tensor(True)

Note that, in `permute()`, you must provide the new order of all the dimensions. In `transpose()`, you can only provide two dimensions. `tranpose()` can be thought as a special case of `permute()` method in for 2D tensors.